In [ ]:
import panda as pd
import statsmodels.api as sm

In [ ]:
# Read data
tdata = pd.read_csv('Titanic_train.csv')

In [ ]:
### Data Preprocessing ###

# Subset data
pdpre = tdata.iloc[:,[1,2,4,5,6,7,9]]
pdpre.head()

# Set ‘male’ to 1, ‘female’ to 0
def sex_to_numeric(x):
    if x == 'male':
        return 1
    if x=='female':
        return 0
pdpre['Sex'] = pdpre['Sex'].apply(sex_to_numeric)

# Data Summary
sumy = pdpre.iloc[:,1:].describe(include='all')

# Two-way contingency table
pd.crosstab(pdpre['Survived'], pdpre['Pclass'], margins=True)

# Missing value
pdpre["Age"].fillna(pdpre["Age"].mean(), inplace=True)

# Transfer categorical variables to dummy variables
pc = pd.get_dummies(pdpre['Pclass'], prefix='Pclass')
sx = pd.get_dummies(pdpre['Sex'], prefix='Sex')
cols_to_keep = ['Survived', 'Age', 'SibSp', 'Parch', 'Fare']
df_temp = pdpre[cols_to_keep].join(pc.loc[: , 'Pclass_2' : ])
df = df_temp.join(sx['Sex_1'])

# Split training and test data
train = df.iloc[ : 802, : ]
test = df.iloc[802 : , : ]

train_y = train['Survived']
train_x = train.iloc[ : , 1 : ]

test_y = test['Survived']
test_x = test.iloc[ : , 1 : ]

In [ ]:
# Fitting Logistic model
train_x['intercept'] = 1.0
logit = sm.Logit(train_y, train_x)
result = logit.fit()
result.summary()

In [ ]:
# Predicting
test_x['intercept'] = 1.0
pred = result.predict(test_x)

# Use 0.5 as the threshold
pred[pred > 0.5] = 1
pred[pred <= 0.5] = 0

# Use the error rate as the evaluation metric
test_err = 1 - (test_y == pred).mean()
print("test error rate: " + str(test_err))